In [1]:
import numpy as np
import cirq
import mitiq
import stim
import stimcirq
import warnings
warnings.filterwarnings("ignore")

In [2]:
n_qubits = 2
num_trials = 10
p_err = 0.004
p_th = 0.009
base_shots = 10**4
device_size = 1500

In [3]:
d_array = np.linspace(27, 5, 12, dtype=int)

In [4]:
def gen_noise_model(p_err, p_th, distance):
    """Create sweepable Pauli noise model."""
    LERR = 0.03 * (p_err / p_th) ** int((distance + 1) / 2)
    return LERR # model as single-qubit errors remaining after correction 

In [5]:
class PauliNoiseModel(cirq.NoiseModel):

    def __init__(self, error_rate):
        self.error_rate = error_rate

    def noisy_operation(self, op):
        error_rate = self.error_rate
        channel = cirq.BitFlipChannel(error_rate).on_each(op.qubits)
        channel += cirq.PhaseFlipChannel(error_rate).on_each(op.qubits)
        return [op, channel]

In [1]:
def fill_circuit(circ):
    filled_circuit = cirq.Circuit()
    qubits = circ.all_qubits()
    for m in circ:
        idle = False
        for q in qubits:
            if not m.operates_on_single_qubit(q):
               idle = True
               filled_circuit.append(m.with_operations(cirq.I(q)))
               break
        if not idle:
          filled_circuit.append(m)

    return filled_circuit

In [20]:
cirq_circuits_depth100 = mitiq.benchmarks.generate_rb_circuits(n_qubits, num_cliffords=100, trials=num_trials)
filled_circuits_depth100 = []
for c in cirq_circuits_depth100:
    filled_circuit = fill_circuit(c)
    filled_circuit.append(cirq.measure(filled_circuit.all_qubits()))
    filled_circuits_depth100.append(filled_circuit)

In [ ]:
noisy_circuits_depth100 = []
stim_circuits_depth100 = []
for di in range(len(d_array)):
    noisy_circuits_depth100 = [f.with_noise(PauliNoiseModel(gen_noise_model(p_err, p_th, d_array[di]))) for f in filled_circuits_depth100[:num_trials]]
    stim_circuits_depth100.append([stimcirq.cirq_circuit_to_stim_circuit(n) for n in noisy_circuits_depth100])

In [7]:
def batched_stim_executor(circ_list, shots):
    noisy_expectation_values = []
    for c in circ_list:
        sampler = c.compile_sampler()
        result = sampler.sample(shots=shots)
        counts = np.count_nonzero([r[0].astype(int) + r[1].astype(int) for r in result])
        noisy_expectation_values.append(1 - counts/shots)
    return noisy_expectation_values

In [8]:
def scale_shots(num_device_qubits, scaled_distance, base_shots, n_qubits_circuit):
    used_qubits = n_qubits_circuit * scaled_distance ** 2
    return base_shots * int(num_device_qubits / used_qubits)

In [13]:
ds_results_depth100 = np.zeros((num_trials, len(d_array)))
for di in range(len(d_array)):
    ds_results_depth100[:, di] = batched_stim_executor(stim_circuits_depth100[di], shots=scale_shots(device_size, d_array[di], base_shots, n_qubits))

np.savetxt("data/ds_depth100.txt", ds_results_depth100)

In [24]:
unmit_results_depth100 = np.zeros((num_trials, len(d_array)-3))
for di in range(len(d_array)-3):
    unmit_results_depth100[:, di] = batched_stim_executor(stim_circuits_depth100[di], shots=scale_shots(device_size, d_array[di], 4*base_shots, n_qubits))

np.savetxt("data/unmit_depth100.txt", unmit_results_depth100)

In [9]:
cirq_circuits_depth1000 = mitiq.benchmarks.generate_rb_circuits(n_qubits, num_cliffords=1000, trials=num_trials)
filled_circuits_depth1000 = []
for c in cirq_circuits_depth1000:
    filled_circuit = fill_circuit(c)
    filled_circuit.append(cirq.measure(filled_circuit.all_qubits()))
    filled_circuits_depth1000.append(filled_circuit)

stim_circuits_depth1000 = [stimcirq.cirq_circuit_to_stim_circuit(f) for f in filled_circuits_depth1000]

In [ ]:
stim_circuits_depth1000 = []
for di in range(len(d_array)):
    noisy_circuits_depth1000 = [f.with_noise(PauliNoiseModel(gen_noise_model(p_err, p_th, d_array[di]))) for f in filled_circuits_depth1000[:num_trials]]
    stim_circuits_depth1000.append([stimcirq.cirq_circuit_to_stim_circuit(n) for n in noisy_circuits_depth1000])

In [31]:
ds_results_depth1000 = np.zeros((num_trials, len(d_array)))
for di in range(len(d_array)):
    ds_results_depth1000[:, di] = batched_stim_executor(stim_circuits_depth1000[di], shots=scale_shots(device_size, d_array[di], base_shots, n_qubits))

np.savetxt("data/ds_depth1000.txt", ds_results_depth1000)

In [32]:
unmit_results_depth1000 = np.zeros((num_trials, len(d_array)-3))
for di in range(len(d_array)-3):
    unmit_results_depth1000[:, di] = batched_stim_executor(stim_circuits_depth1000[di], shots=scale_shots(device_size, d_array[di], 4*base_shots, n_qubits))

np.savetxt("data/unmit_depth1000.txt", unmit_results_depth1000)

In [8]:
cirq_circuits_depth10k = mitiq.benchmarks.generate_rb_circuits(n_qubits, num_cliffords=10**4, trials=num_trials)
filled_circuits_depth10k = []
for c in cirq_circuits_depth10k:
    filled_circuit = fill_circuit(c)
    filled_circuit.append(cirq.measure(filled_circuit.all_qubits()))
    filled_circuits_depth10k.append(filled_circuit)

In [ ]:
stim_circuits_depth10k = []
for di in range(len(d_array)):
    noisy_circuits_depth10k = [f.with_noise(PauliNoiseModel(gen_noise_model(p_err, p_th, d_array[di]))) for f in filled_circuits_depth10k[:num_trials]]
    stim_circuits_depth10k.append([stimcirq.cirq_circuit_to_stim_circuit(n) for n in noisy_circuits_depth10k])

In [11]:
ds_results_depth10k = np.zeros((num_trials, len(d_array)))
for di in range(len(d_array)):
    ds_results_depth10k[:, di] = batched_stim_executor(stim_circuits_depth10k[di], shots=scale_shots(device_size, d_array[di], base_shots, n_qubits))

np.savetxt("data/ds_depth10000.txt", ds_results_depth10k)

In [16]:
unmit_results_depth10k = np.zeros((num_trials, 3))
for di in range(len(d_array) - 3):
    unmit_results_depth10k[:, di] = batched_stim_executor(stim_circuits_depth10k[di], shots=scale_shots(device_size, d_array[di], 4*base_shots, n_qubits))

np.savetxt("data/unmit_depth10000.txt", unmit_results_depth10k)

In [10]:
scale_factors = [1, 3, 5, 7]

In [11]:
def add_noise_to_stim_circuits(circuits, p_err, p_th, d):
    noisy_stim_circuits = []
    for c in range(len(circuits)):
        noisy = stim.Circuit()
        stim_circ = circuits[c]
        for instruction in stim_circ:
            noisy.append(instruction)
            if instruction.name != "TICK":
                noisy.append("X_ERROR", instruction.targets_copy(), gen_noise_model(p_err, p_th, d))
                noisy.append("Z_ERROR", instruction.targets_copy(), gen_noise_model(p_err, p_th, d))
        noisy_stim_circuits.append(noisy)
    return noisy_stim_circuits

In [ ]:
stim_folded_circuits_depth100 = []
for s in scale_factors[1:]:
    folded_circuits_depth100 = [mitiq.zne.scaling.fold_global(c, s) for c in filled_circuits_depth100]
    stim_folded_circuits_depth100.append([stimcirq.cirq_circuit_to_stim_circuit(f) for f in folded_circuits_depth100])

In [ ]:
for d in d_array[:9]:
    results = np.zeros((num_trials, 3))
    for s in range(3):
        noisy_circuits = add_noise_to_stim_circuits(stim_folded_circuits_depth100[s], p_err, p_th, d)
        shots = scale_shots(device_size, d, base_shots, n_qubits)
        results[:, s] = batched_stim_executor(noisy_circuits, shots)
    np.savetxt(f"/Users/mistywahl/Documents/GitHub/mitiq-internal/ds-zne/data/folding_depth100_d{d}", results)

In [ ]:
stim_folded_circuits_depth1000 = []
for s in scale_factors[1:]:
    folded_circuits_depth1000 = [mitiq.zne.scaling.fold_global(c, s) for c in filled_circuits_depth1000]
    stim_folded_circuits_depth1000.append([stimcirq.cirq_circuit_to_stim_circuit(f) for f in folded_circuits_depth1000])

In [ ]:
for d in d_array[:9]:
    results = np.zeros((num_trials, 3))
    for s in range(3):
        noisy_circuits = add_noise_to_stim_circuits(stim_folded_circuits_depth1000[s], p_err, p_th, d)
        shots = scale_shots(device_size, d, base_shots, n_qubits)
        results[:, s] = batched_stim_executor(noisy_circuits, shots)
    np.savetxt(f"/Users/mistywahl/Documents/GitHub/mitiq-internal/ds-zne/data/folding_depth1000_d{d}", results)

In [2]:
stim_folded_circuits_depth10k = []
for s in scale_factors[1:]:
    folded_circuits_depth10k = [mitiq.zne.scaling.fold_global(c, s) for c in filled_circuits_depth10k]
    stim_folded_circuits_depth10k.append([stimcirq.cirq_circuit_to_stim_circuit(f) for f in folded_circuits_depth10k])

In [ ]:
for d in d_array[:9]:
    results = np.zeros((num_trials, 3))
    for s in range(3):
        noisy_circuits = add_noise_to_stim_circuits(stim_folded_circuits_depth10k[s], p_err, p_th, d)
        shots = scale_shots(device_size, d, base_shots, n_qubits)
        results[:, s] = batched_stim_executor(noisy_circuits, shots)
    np.savetxt(f"data/folding_depth10000_d{d}", results)